In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import seaborn as sns
from torch.utils.data import DataLoader
import os
import sys
import re
project_root = os.path.dirname(os.path.dirname(os.path.abspath('Pheno.ipynb')))
src_path = os.path.join(project_root, 'src')
sys.path.append(src_path)
%load_ext autoreload 
%autoreload 2

### Preprocess Liver Cancer datasets

In [1]:
def find_csv_filenames(path_to_dir, string_in_filename):
    filenames = os.listdir(path_to_dir)
    return [filename for filename in filenames if all([
        filename.endswith('.csv'),
        string_in_filename in filename
    ])]

# 示例用法
path = '../data/Liver'
string = 'cell_by_gene'
csv_files = sorted(find_csv_filenames(path, string))
print(csv_files)
metadata_files = sorted(find_csv_filenames(path, 'metadata'))
print(metadata_files)

NameError: name 'os' is not defined

In [8]:
exp_matrix = pd.read_csv(os.path.join(path, csv_files[i]), index_col=0)
exp_matrix

,ACTA2,ACTG2,ADAM12,ADAM28,ADGRE5,ADH1B,CCL21,AIM2,ANKRD55,AREG,...,Blank-51,Blank-52,Blank-53,Blank-54,Blank-55,Blank-56,Blank-57,Blank-58,Blank-59,Blank-60
cell,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,4.0,7.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,5.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131421,1.0,0.0,0.0,0.0,4.0,2.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131422,0.0,0.0,0.0,0.0,3.0,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
regex = re.compile(r'(\d+)_region_(\d+)')
# 读取数据
pseudo_bulk = {}
for i in range(len(csv_files)):
    exp_matrix = pd.read_csv(os.path.join(path, csv_files[i]), index_col=0)
    if i == 0:
       gene_features = exp_matrix.columns 
    exp_matrix = exp_matrix.loc[:, gene_features]
    print(exp_matrix.columns)
    metadata = pd.read_csv(os.path.join(path, metadata_files[i]), index_col=0)
    match = regex.search(csv_files[i])
    if match:
        adata_filename = f'{match.group(1)}_region_{match.group(2)}.h5ad'
        print(adata_filename)
    adata = anndata.AnnData(X=exp_matrix.values, obs=metadata)
    print(adata)
    pseudo_bulk[f'{match.group(1)}_{match.group(2)}'] = adata.X.sum(axis=0)
    adata.write_h5ad(os.path.join(path, adata_filename))

Index(['ACTA2', 'ACTG2', 'ADAM12', 'ADAM28', 'ADGRE5', 'ADH1B', 'CCL21',
       'AIM2', 'ANKRD55', 'AREG',
       ...
       'Blank-51', 'Blank-52', 'Blank-53', 'Blank-54', 'Blank-55', 'Blank-56',
       'Blank-57', 'Blank-58', 'Blank-59', 'Blank-60'],
      dtype='object', length=461)
1003_region_0.h5ad
AnnData object with n_obs × n_vars = 369399 × 461
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount'


/raid1/YiDingcheng/miniforge3/envs/pyg/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['ACTA2', 'ACTG2', 'ADAM12', 'ADAM28', 'ADGRE5', 'ADH1B', 'CCL21',
       'AIM2', 'ANKRD55', 'AREG',
       ...
       'Blank-51', 'Blank-52', 'Blank-53', 'Blank-54', 'Blank-55', 'Blank-56',
       'Blank-57', 'Blank-58', 'Blank-59', 'Blank-60'],
      dtype='object', length=461)
1012_region_0.h5ad
AnnData object with n_obs × n_vars = 283525 × 461
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount'


/raid1/YiDingcheng/miniforge3/envs/pyg/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['ACTA2', 'ACTG2', 'ADAM12', 'ADAM28', 'ADGRE5', 'ADH1B', 'CCL21',
       'AIM2', 'ANKRD55', 'AREG',
       ...
       'Blank-51', 'Blank-52', 'Blank-53', 'Blank-54', 'Blank-55', 'Blank-56',
       'Blank-57', 'Blank-58', 'Blank-59', 'Blank-60'],
      dtype='object', length=461)
1012_region_1.h5ad
AnnData object with n_obs × n_vars = 131426 × 461
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount'


...

/raid1/YiDingcheng/miniforge3/envs/pyg/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['ACTA2', 'ACTG2', 'ADAM12', 'ADAM28', 'ADGRE5', 'ADH1B', 'CCL21',
       'AIM2', 'ANKRD55', 'AREG',
       ...
       'Blank-51', 'Blank-52', 'Blank-53', 'Blank-54', 'Blank-55', 'Blank-56',
       'Blank-57', 'Blank-58', 'Blank-59', 'Blank-60'],
      dtype='object', length=461)
122_region_1.h5ad


/raid1/YiDingcheng/miniforge3/envs/pyg/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 151647 × 461
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount'
Index(['ACTA2', 'ACTG2', 'ADAM12', 'ADAM28', 'ADGRE5', 'ADH1B', 'CCL21',
       'AIM2', 'ANKRD55', 'AREG',
       ...
       'Blank-51', 'Blank-52', 'Blank-53', 'Blank-54', 'Blank-55', 'Blank-56',
       'Blank-57', 'Blank-58', 'Blank-59', 'Blank-60'],
      dtype='object', length=461)
63_region_0.h5ad
AnnData object with n_obs × n_vars = 228679 × 461
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount'


/raid1/YiDingcheng/miniforge3/envs/pyg/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
pseudo_bulk = pd.DataFrame(pseudo_bulk)
pseudo_bulk.index = exp_matrix.columns
pseudo_bulk

,1003_0,1012_0,1012_1,1014_0,1014_1,1017_0,1029_0,122_0,122_1,63_0
ACTA2,270227.0,434886.0,201380.0,59374.0,9884.0,32705.0,106966.0,38842.0,22886.0,13156.0
ACTG2,22833.0,17338.0,5980.0,3665.0,554.0,1138.0,5193.0,3040.0,3080.0,1697.0
ADAM12,16661.0,20374.0,3441.0,1697.0,399.0,3404.0,3379.0,5146.0,6441.0,2016.0
ADAM28,35283.0,13956.0,12717.0,1170.0,88.0,1554.0,3660.0,4583.0,3822.0,1905.0
ADGRE5,402208.0,346434.0,87069.0,41693.0,7982.0,42169.0,47220.0,60694.0,35784.0,29150.0
...,...,...,...,...,...,...,...,...,...,...
Blank-56,5819.0,2860.0,1556.0,388.0,52.0,214.0,555.0,520.0,652.0,279.0
Blank-57,5225.0,3626.0,1504.0,408.0,58.0,502.0,607.0,736.0,625.0,489.0
Blank-58,5817.0,2875.0,1964.0,427.0,54.0,253.0,797.0,1120.0,821.0,419.0
Blank-59,17939.0,7644.0,3699.0,888.0,75.0,538.0,1721.0,726.0,594.0,675.0


In [5]:
pseudo_bulk.to_csv(os.path.join(path, 'pseudo_bulk.csv'))

In [25]:
with open('Liver/genenames.txt', 'w') as f:
    for gene in exp_matrix.columns:
        f.write(f'{gene}\n')

In [11]:
data = pd.read_csv('../data/Cleaned_data/Liver/TCGA-LIHC.htseq_counts_clean_tumor.csv', index_col=0)
data

,ACTA2,ACTG2,ADAM12,ADAM28,ADGRE5,ADH1B,AIM2,ANKRD55,AREG,ASPN,...,VPS37B,VWF,WDFY4,XBP1,XCR1,YPEL5,ZBTB16,ZC3HAV1,ZFP36L1,ZNF331
TCGA.FV.A495.01A,6094,96,65,69,1726,21991,16,19,42,1500,...,622,3707,153,0,22,1774,623,777,6809,349
TCGA.ED.A7PZ.01A,8413,1476,55,11,4548,3495,3,10,218,345,...,675,3256,25,0,1,3286,13,858,2832,1024
TCGA.ED.A97K.01A,6415,109,239,5451,3765,27048,40,45,252,1551,...,2616,2556,294,0,13,6159,81,2871,15971,176
TCGA.ED.A7PX.01A,18077,1569,598,155,2888,540,13,10,50,2270,...,1682,3415,180,0,15,5204,23,1123,8968,430
TCGA.BC.A3KF.01A,4487,619,54,47,554,17776,1,36,7,201,...,621,3903,104,0,22,7001,427,2236,7918,1211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.DD.A3A6.01A,36377,626,6470,34,2377,6564,8,2,54,199,...,1463,7794,183,0,29,1973,884,630,3767,355
TCGA.DD.A4NP.01A,5408,314,11,23,643,423517,5,25,7,515,...,596,4764,85,0,7,5351,422,1788,9308,94
TCGA.DD.A118.01A,10327,176,146,48,927,1466,7,3,23,1141,...,388,8270,160,0,13,7931,188,1081,6480,1327
TCGA.DD.A73G.01A,4369,464,13,26,1141,5949,3,23,7,418,...,723,9429,65,0,1,2410,29,694,3178,79


In [37]:
clinical = pd.read_table('TCGA/TCGA-LIHC.GDC_phenotype.tsv')
clinical.columns

Index(['submitter_id.samples',
       'adjacent_hepatic_tissue_inflammation_extent_type',
       'age_at_initial_pathologic_diagnosis', 'albumin_result_lower_limit',
       'albumin_result_specified_value', 'albumin_result_upper_limit',
       'batch_number', 'bcr', 'bcr_followup_barcode', 'bcr_followup_uuid',
       ...
       'days_to_collection.samples', 'days_to_sample_procurement.samples',
       'initial_weight.samples', 'is_ffpe.samples', 'oct_embedded.samples',
       'preservation_method.samples', 'sample_type.samples',
       'sample_type_id.samples', 'state.samples', 'tissue_type.samples'],
      dtype='object', length=119)

In [38]:
clinical[['Pathologic']]

,age_at_initial_pathologic_diagnosis,albumin_result_lower_limit,albumin_result_specified_value,albumin_result_upper_limit,bilirubin_lower_limit,bilirubin_upper_limit,cancer_first_degree_relative,creatinine_lower_level,creatinine_upper_limit,creatinine_value_in_mg_dl,...,days_to_last_follow_up.diagnoses,year_of_diagnosis.diagnoses,bmi.exposures,height.exposures,weight.exposures,days_to_collection.samples,days_to_sample_procurement.samples,initial_weight.samples,preservation_method.samples,sample_type_id.samples
count,468.000000,369.000000,378.000000,369.000000,376.000000,386.000000,171.000000,382.000000,384.000000,384.000000,...,329.000000,466.000000,417.000000,422.000000,433.000000,466.000000,0.0,469.000000,0.0,469.000000
mean,60.264957,14.130623,18.081481,19.265583,0.129521,1.035233,1.771930,1.699215,3.157031,2.614323,...,842.914894,2008.845494,26.634508,167.635071,74.094688,1272.504292,NaN,265.162047,NaN,2.904051
std,13.834533,197.656825,267.283631,265.264388,0.096917,1.770143,1.418585,7.324469,12.876252,11.335129,...,776.564828,4.285227,9.646802,11.528937,19.758092,1349.031291,NaN,331.229204,NaN,3.923053
min,16.000000,0.300000,0.200000,0.500000,0.000000,0.100000,0.000000,0.000000,0.900000,0.400000,...,0.000000,1995.000000,14.526644,64.000000,40.000000,7.000000,NaN,1.000000,NaN,1.000000
25%,52.000000,3.500000,3.500000,5.000000,0.100000,0.500000,1.000000,0.600000,1.200000,0.800000,...,347.000000,2007.000000,21.877551,162.000000,61.000000,170.750000,NaN,80.000000,NaN,1.000000
50%,62.000000,3.500000,4.000000,5.000000,0.100000,0.700000,1.000000,0.700000,1.400000,0.900000,...,615.000000,2010.000000,24.983563,168.000000,71.000000,777.500000,NaN,160.000000,NaN,1.000000
75%,70.000000,3.800000,4.300000,5.200000,0.200000,1.000000,2.000000,0.800000,1.500000,1.100000,...,1145.000000,2012.000000,29.320988,174.000000,84.000000,2025.500000,NaN,290.000000,NaN,1.000000
max,90.000000,3800.000000,5200.000000,5100.000000,1.000000,19.000000,9.000000,62.000000,120.000000,124.000000,...,3675.000000,2013.000000,131.835938,196.000000,172.000000,6014.000000,NaN,2190.000000,NaN,11.000000


In [3]:
expression_data = pd.read_csv('../data/Cleaned_data/Liver/TCGA-LIHC.htseq_counts_clean_tumor.csv', index_col=0)
label = pd.read_csv('../data/Cleaned_data/Liver/TCGA-LIHC.survival_clean_tumor.csv', index_col=0)

In [8]:
train_exp.shape

(294, 398)

In [4]:
from sklearn.model_selection import train_test_split
train_exp, test_exp, train_label, test_label = train_test_split(expression_data, label, test_size=0.2, random_state=42)
train_exp.to_csv('../data/Cleaned_data/Liver/train_exp.csv')
test_exp.to_csv('../data/Cleaned_data/Liver/test_exp.csv')
train_label.to_csv('../data/Cleaned_data/Liver/train_label.csv')
test_label.to_csv('../data/Cleaned_data/Liver/test_label.csv')

In [2]:
from dataset import TCGADataset
train_set = TCGADataset('../data/Cleaned_data/Liver/train_exp.csv', '../data/Cleaned_data/Liver/train_label.csv', 'median_surv')

In [3]:
label = pd.read_csv('../data/Cleaned_data/Liver/train_label.csv', index_col=0)

In [4]:
label['median_surv'].astype('category').cat.codes

TCGA.ZS.A9CF.01A    0
TCGA.G3.A25T.01A    1
TCGA.MI.A75C.01A    1
TCGA.UB.AA0V.01A    1
TCGA.DD.AAVR.01A    0
                   ..
TCGA.CC.A5UE.01A    1
TCGA.FV.A3R3.01A    1
TCGA.DD.AAEK.01A    1
TCGA.ZS.A9CF.02A    0
TCGA.CC.A3MC.01A    1
Length: 294, dtype: int8

In [5]:
def collate_fn(batch):
    data_batch, target_batch, batch_index = zip(*batch)

    batch_index = torch.tensor(batch_index, dtype=torch.float32).view(-1, 1)
    data_batch = torch.stack(data_batch, dim=0)
    target_batch = torch.stack(target_batch, dim=0)
    library_sizes = torch.sum(data_batch, dim=1)

    mean = torch.mean(library_sizes).expand(data_batch.size(0))
    variance = torch.var(library_sizes).expand(data_batch.size(0))

    return data_batch, target_batch, batch_index, mean, variance

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [6]:
for data, target, batch_index, mean, variance in train_loader:
    print(data, target, batch_index, mean, variance)
    break

tensor([[3.7930e+03, 2.7700e+02, 3.6900e+02,  ..., 2.9140e+03, 2.0447e+04,
         2.1310e+03],
        [3.5310e+03, 1.0000e+02, 8.0000e+00,  ..., 3.8300e+02, 3.7090e+03,
         4.6000e+01],
        [9.9860e+03, 2.7000e+02, 2.3000e+01,  ..., 1.2590e+03, 5.8960e+03,
         2.9900e+02],
        ...,
        [3.4380e+03, 2.1000e+02, 2.3000e+01,  ..., 1.0360e+03, 4.4810e+03,
         5.6300e+02],
        [1.2386e+04, 3.7500e+02, 1.6400e+02,  ..., 2.4140e+03, 5.2720e+03,
         1.8730e+03],
        [2.4230e+03, 8.3000e+01, 9.7000e+01,  ..., 9.0600e+02, 2.8017e+04,
         1.1490e+03]]) tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        

In [8]:
n_input = 398
n_hidden = 64
n_latent = 32
n_batch = len(train_set)
n_labels = 2
print(n_batch)

294


In [9]:
from model import BulkVAE

model = BulkVAE(n_input, n_batch, n_labels, n_hidden, n_latent)

optimizer = torch.optim.Adam(model.parameters(), lr=1)

for batch in train_loader:
    optimizer.zero_grad()
    data_batch, target_batch, batch_index, mean, variance = batch
    recon_loss, kl_z, kl_l, classification_loss, probs = model(data_batch, mean, variance, batch_index, target_batch)
    total_loss = torch.mean(recon_loss + kl_z + kl_l) + classification_loss
    total_loss.backward()
    optimizer.step()
    print(recon_loss, kl_z, kl_l, classification_loss, total_loss)
    break


tensor([10702743.0000,  4751273.0000, 25516744.0000, 12944392.0000,
        11689364.0000, 19038540.0000,  7409312.0000, 12891787.0000,
        16920158.0000, 25252558.0000,  4895038.0000, 24234324.0000,
         5555599.0000,  7734252.0000, 17018790.0000, 17231220.0000,
         6446980.0000, 20891880.0000, 11010207.0000,  7049693.0000,
        27267102.0000,  9580864.0000,  7624444.0000, 11348685.0000,
        12672920.0000, 16916026.0000,  6989196.5000, 12427053.0000,
        17487262.0000, 12102446.0000,  7138230.5000, 13046132.0000],
       grad_fn=<NegBackward0>) tensor([ 7.6726,  1.2526,  3.4310,  2.7497,  2.3621,  7.7888,  2.6538,  5.2459,
        19.8198,  1.9583,  5.8649, 13.8285,  1.5784,  4.6270,  0.6725,  6.5620,
         9.8480,  5.3109,  4.0237,  1.3213,  4.2406,  9.2825,  3.8794,  2.4801,
         9.8914, 15.9775,  2.1207,  0.8508,  6.8058,  4.2357,  0.3519,  5.2665],
       grad_fn=<SumBackward1>) tensor([863.2360, 863.2360, 863.2360, 859.4595, 835.7320, 839.1875, 863.

In [16]:
def _convert_mean_disp_to_counts_logits(mu, theta, eps=1e-6):
    r"""NB parameterizations conversion

        :param mu: mean of the NB distribution.
        :param theta: inverse overdispersion.
        :param eps: constant used for numerical log stability.
        :return: the number of failures until the experiment is stopped
            and the success probability.
    """
    assert (mu is None) == (
        theta is None
    ), "If using the mu/theta NB parameterization, both parameters must be specified"
    logits = (mu + eps).log() - (theta + eps).log()
    total_count = theta
    return total_count, logits

# Negative Binomail

In [24]:
from distribution import NegativeBinomial
from torch.distributions import NegativeBinomial as NB
tens = torch.Tensor([1000, 2000, 1310, 4203, 2310])
mu = torch.Tensor([1000])
theta = torch.Tensor([2000])
dist = NegativeBinomial(mu=mu, theta=theta)
total_count, logits = _convert_mean_disp_to_counts_logits(mu, theta)
print(total_count, logits)
dist_t = NB(total_count=total_count, logits=logits)
print(dist.log_prob(tens))
print(dist_t.log_prob(tens))


tensor([2000.]) tensor([-0.6931])
tensor([[   -4.5771,  -240.6323,   -33.0034, -1534.3276,  -377.5938]])
tensor([   -4.5776,  -240.6323,   -33.0029, -1534.3296,  -377.5938])
